In [185]:
import numpy as np
from scipy import signal

In [186]:
img = np.load("sample.npy")
img = np.floor(img/2)
filt = np.load("filter.npy")

### Defining the convolution with and without our lookup based multiply 

In [187]:
def conv2d(a, b):
    a = np.array(a)
    b = np.array(b)
    
    a_shape = np.shape(a)
    b_shape = np.shape(b)
    
    a_b_diff_dim_0 = np.abs(a_shape[0]-b.shape[0]+1)
    a_b_diff_dim_1 = np.abs(a_shape[1]-b.shape[1]+1)
    
    conv_res = np.zeros([a_b_diff_dim_0, a_b_diff_dim_1])
    
    for i in range(a_b_diff_dim_0):
        for j in range(a_b_diff_dim_1):
            conv_res[i,j] = np.sum(np.multiply(np.flip(b), a[i:i + b_shape[0], j:j + b_shape[1]]))
    
    return conv_res


def create_lookup_table():
    Multiplier = np.zeros((256, 256), dtype=int)
    for i in range(256):
        for j in range(256):
            Multiplier[i, j] = (i - 128) * (j - 128)
    return Multiplier


def My_Mult(a, b):
    assert a.shape == b.shape, "Input matrices must have the same shape."
    a_int = a.astype(int)
    b_int = b.astype(int)
    result = Multiplier[a_int + 128, b_int + 128]
    return result


def MIMIconv2d(a, b):
    a = np.array(a)
    b = np.array(b)
    
    a_shape = np.shape(a)
    b_shape = np.shape(b)
    
    a_b_diff_dim_0 = np.abs(a_shape[0]-b.shape[0]+1)
    a_b_diff_dim_1 = np.abs(a_shape[1]-b.shape[1]+1)
    
    conv_res = np.zeros([a_b_diff_dim_0, a_b_diff_dim_1])
    
    for i in range(a_b_diff_dim_0):
        for j in range(a_b_diff_dim_1):
            conv_res[i,j] = np.sum(My_Mult(np.flip(b), a[i:i + b_shape[0], j:j + b_shape[1]]), dtype=int)
    
    return conv_res


In [190]:
Multiplier = create_lookup_table()

mimi_result = MIMIconv2d(img, filt)
fofo = signal.convolve2d(img,filt, mode = "valid")
foo = conv2d(img, filt)

print(np.shape(my_result)[0])
print(np.shape(foo)[0])
print(np.shape(fofo)[0])

error = np.sum(np.abs(fofo - mimi_result))
print(f"Error: {error}")

254
254
254
Error: 0.0


With create_lookup_table() we have initialized the lookuptable for al mulitplications of integers from 0 to 255.

My_Mult uses this to look up the result of the multiplication. Unfortunately, while we are saving computational complexity by simply looking up our muliplication results in a precomputed table, we are introducing additional memory complexity which could also hurt our performance, especially if we are not compute but memory bound and our problem size exceeds the size of our cache. 

In the fourth subtask we simply replace the numpy multiply with our own freshly implemented one.
I would have expected for the accuracy to go down, but either our input values are not extreme enough to have a considerable error from the quantization or I might have done something wrong.